In [5]:

import pandas as pd
import langid
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import en_core_web_sm
import spacy_fastlang
import regex

nlp = en_core_web_sm.load()
nlp.add_pipe("language_detector")




In [6]:
#setting up necessary proccessing things
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()


df = pd.read_csv('data\goodreads_nonfiction_books.csv')
df['Review'] = df['Review'].str.replace("121", " ")
df['Review'] = df['Review'].str.replace("\n", " ")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ieyua\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ieyua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ieyua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def remove_non_latin_letters(text): #removes anything other than non latin characters
    cleaned_text = ''.join(c if (regex.match(r'\p{IsLatin}', c) or c.isdigit() or c.isspace() or (c == "'" and i < len(text) - 1 and regex.match(r'\p{IsLatin}', text[i+1]))) else ' ' for i, c in enumerate(str(text)))
    return ' '.join(cleaned_text.split())

def chunk_text (text, chunk_size=75):
    words = text.split()
    for i in range (0, len(words), chunk_size):
        yield " ".join(words[i:i+chunk_size])

def check_en (text):
    doc = nlp(text)
    return doc._.language == 'en'

def process_item(text):
    processed_text = []
    for chunk in chunk_text(text):
        if check_en(chunk):
            processed_text.append(chunk)
    return ' '.join(processed_text)



In [8]:
#remove all none letters from reviews synopsis and genres
columns_to_clean = ['Synopsis','Review','Genres']
df["Raw Synopsis"] = df["Synopsis"]
for column in columns_to_clean:
    df[column] = df[column].apply(remove_non_latin_letters)
    
for column in columns_to_clean:
    try:
        df[column] = df[column].str.lower()
    except:
        print ("error")


#remove all the failed data from webscraping
df_filtered = df[~df.eq("errorlololol").any(axis=1)]
df = df_filtered

df
columns_to_check = ['Review', 'Genres', 'Synopsis']

for col in columns_to_check:
    if df[col].isna().any():
        print(f"'{col}' column contains NoneType or NaN values.")
    else:
        print(f"'{col}' column does not contain NoneType or NaN values.")

df

'Review' column does not contain NoneType or NaN values.
'Genres' column does not contain NoneType or NaN values.
'Synopsis' column does not contain NoneType or NaN values.


,Book Title,Author,Average Rating,Total Rating,Publish Date,Synopsis,Review,Genres,Raw Synopsis
0,The Diary of a Young Girl (Mass Market Paperback),Anne Frank,4.19,3707727,1947,discovered in the attic in which she spent the...,why do we write reviews you have a lot of reas...,'classics 'nonfiction 'biography 'memoir 'hist...,Discovered in the attic in which she spent the...
1,Sapiens: A Brief History of Humankind (Paperback),Yuval Noah Harari,4.36,1016030,2011,from a renowned historian comes a groundbreaki...,sapiens a brief history of humankind is a book...,'nonfiction 'history 'science 'philosophy 'ant...,From a renowned historian comes a groundbreaki...
2,Educated (Hardcover),Tara Westover,4.47,1507027,2018,tara westover was 17 the first time she set fo...,on the highway below the school bus rolls past...,'nonfiction 'memoir 'biography 'audiobook 'bio...,Tara Westover was 17 the first time she set fo...
3,Quiet: The Power of Introverts in a World That...,Susan Cain,4.08,432715,2012,the book that started the quiet revolutionat l...,there's zero correlation between being the bes...,'nonfiction 'psychology 'self help 'business '...,The book that started the Quiet RevolutionAt l...
4,The Immortal Life of Henrietta Lacks (Hardcover),Rebecca Skloot,4.11,731428,2010,her name was henrietta lacks but scientists kn...,the doorbell rang the other day and when i ans...,'nonfiction 'science 'history 'biography 'medi...,"Her name was Henrietta Lacks, but scientists k..."
...,...,...,...,...,...,...,...,...,...
1245,Counting the Cost (Kindle Edition),Jill Duggar,4.18,75053,2023,for the first time discover the unedited truth...,full video explaining a lot more of my thought...,'nonfiction 'memoir 'audiobook 'biography 'rel...,"For the first time, discover the unedited trut..."
1246,The Theory of Everything: The Origin and Fate ...,Stephen Hawking,4.18,24736,2002,based on a series of lectures given at cambrid...,the theory of everything the origin and fate o...,'science 'nonfiction 'physics 'audiobook 'astr...,Based on a series of lectures given at Cambrid...
1247,The Book of Eels: Our Enduring Fascination wit...,Patrik Svensson,3.92,17630,2019,remarkably little is known about the european ...,this book should be titled the book of eels a ...,'nonfiction 'science 'nature 'memoir 'animals ...,Remarkably little is known about the European ...
1248,Fluent Forever: How to Learn Any Language Fast...,Gabriel Wyner,4.09,11081,2014,national bestseller for anyone who wants to le...,i am amazed that so much first hand insight in...,'nonfiction 'language 'self help 'linguistics ...,NATIONAL BESTSELLER • For anyone who wants to ...


In [12]:
columns_to_proccess = ['Synopsis', 'Review']


    
df['Proccessed Synopsis'] = df['Synopsis'].apply(process_item)
df['Proccessed Review'] = df['Review'].apply(process_item)

    
df
    

KeyboardInterrupt: 

In [51]:
with open("text.txt", "w") as file:
    file.write(f"{df['Proccessed Synopsis']}")
pd.set_option('display.max_rows',1250)

        
df.drop(index=[1148,736], inplace=True)
print(df["Proccessed Synopsis"])
#1148
#736
        
print(len(df))


0       discovered in the attic in which she spent the...
1       from a renowned historian comes a groundbreaki...
2       tara westover was 17 the first time she set fo...
3       the book that started the quiet revolutionat l...
4       her name was henrietta lacks but scientists kn...
5       which is more dangerous a gun or a swimming po...
6       learn what sets high achievers apart from bill...
7       in a life filled with meaning and accomplishme...
8       author erik larson imbues the incredible event...
9       no matter your goals atomic habits offers a pr...
10      bill bryson describes himself as a reluctant t...
11      what does feminism mean today that is the ques...
12      a heartbreaking and hilarious memoir by icarly...
13      alternate cover edition of isbn 9780062457738i...
14      born in the town of sighet transylvania elie w...
15      krakauer s page turning bestseller explores a ...
16      psychiatrist viktor frankl's memoir has rivete...
17      the me

In [47]:
df.to_csv('data\cleaned_output_file2.csv', index=False)